## Please be more open minded open exploiting efficient sparse lookups.

The `gurobipy.tupledict.prod` function really is great, but I honestly think you are being far too inefficient in how you exploit a dictish coefs argument. In fact, I find this inefficiency so far outside of "least astonishent" as to be a bug (albeit a minor one) but of course that is for you to decide. 

In [1]:
import gurobipy as gu

In [2]:
mdl = gu.Model()
super_vars = mdl.addVars([(1,i) for i in range(3,4000)] +  
                     [(2,i) for i in range(5,4000)] + 
                     [(3,i) for i in range(2,4000)], 
                     vtype=gu.GRB.BINARY,name='super_vars')
def yourSlicing():
    rtn = []
    for i,j in enumerate([1, 2, 3]):
        coefs = {(j, j*10): 10, (j, j*20):20, (j, j*30):30}
        rtn.append(super_vars.prod(coefs) <= j)
    return rtn

In [3]:
%timeit yourSlicing()

10 loops, best of 3: 63.2 ms per loop


In [4]:
mdl = gu.Model()
super_vars = mdl.addVars([(1,i) for i in range(3,4000)] +  
                     [(2,i) for i in range(5,4000)] + 
                     [(3,i) for i in range(2,4000)], 
                     vtype=gu.GRB.BINARY,name='super_vars')
def mySlicing():
    rtn = []
    for i,j in enumerate([1, 2, 3]):
        coefs = {(j, j*10): 10, (j, j*20):20, (j, j*30):30}
        rtn.append(gu.quicksum(v*super_vars[k] for k,v in coefs.items()) <= j)
    return rtn

In [5]:
%timeit mySlicing()

10000 loops, best of 3: 28.3 µs per loop


To my mind, the `mySlicing` look-up method is simply what I would expect `prod` to do when passed a dict. After all, `gurobipy.tupledict` is really a `dict`, and the pythonic expectation is that when two dict's cross product with each other they do it efficiently by exploitng the efficient hash-based indexing.

## Extended examination

Here is a tickier version along these lines. I possibly wandered into absurd circumstances here, so if your technical fix doesn't address this I wouldn't worry too much... just curious as to how it runs. 

In [6]:
mdl = gu.Model()
super_vars = mdl.addVars([(j,i) for i in range(3,5000) for j in range(1,10)], 
                     vtype=gu.GRB.BINARY,name='super_vars')
def yourSlicing():
    rtn = []
    for i,j in enumerate(range(2000)):
        coefs = {(j%10, j*10): 10, (j%10, (j*20)%5000):20, (j%10, (j*30)%5000):30, 
                 (1,7):100, (2,8):200, (3,9):300}
        rtn.append(super_vars.prod(coefs, j%10, '*') <= j)
    return rtn

In [7]:
%timeit yourSlicing()

1 loops, best of 3: 17.6 s per loop


In [8]:
mdl = gu.Model()
super_vars = mdl.addVars([(j,i) for i in range(3,5000) for j in range(1,10)], 
                     vtype=gu.GRB.BINARY,name='super_vars')
def mySlicing():
    rtn = []
    tl = gu.tuplelist(super_vars)
    selects = {}
    for i,j in enumerate(range(2000)):
        coefs = {(j%10, j*10): 10, (j%10, (j*20)%5000):20, (j%10, (j*30)%5000):30, 
                 (1,7):100, (2,8):200, (3,9):300}
        if j%10 not in selects:
            selects[j%10] = set(tl.select(j%10, '*'))
        rtn.append(gu.quicksum(coefs[k]*super_vars[k] for k in selects[j%10].intersection(coefs)) <= j)
    return rtn

In [9]:
%timeit mySlicing()

10 loops, best of 3: 104 ms per loop
